In [1]:
import pandas as pd
import numpy
import geopy.distance
import math
import sys

In [2]:
dataset = pd.read_csv("preprocessed_dataset.csv")

In [3]:
# Calculate the distance between the two points
# Return a binary value for match/mismatch


def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    if dist > 30:
        return -1
    else:
        return 1

In [4]:
# Create similarity matrix between two tracks

gap_penalty = -1


def create_matrix(track1, track2):
    matrix = numpy.zeros((len(track1) + 1, len(track2) + 1))
    for i in range(0, len(track1) + 1):
        matrix[i, 0] = gap_penalty * i
    for j in range(0, len(track2) + 1):
        matrix[0, j] = gap_penalty * j

    for i in range(1, len(track1) + 1):
        t1 = track1[i - 1]
        for j in range(1, len(track2) + 1):
            t2 = track2[j - 1]
            match = matrix[i - 1, j - 1] + point_distance(t1, t2)
            delete = matrix[i - 1, j] + gap_penalty
            insert = matrix[i, j - 1] + gap_penalty
            matrix[i, j] = max(match, max(delete, insert))

    return matrix

In [5]:
def backtrack(matrix, track1, track2):

    i = len(track1)
    j = len(track2)

    a1 = []

    while i > 0 and j > 0:
        if (
            matrix[i, j] == matrix[i - 1, j - 1] + 1
            and point_distance(
                (track1[i - 1][0], track1[i - 1][1]),
                (track2[j - 1][0], track2[j - 1][1]),
            )
            > 0
        ):
            a1.insert(0, [track1[i - 1][0], track1[i - 1][1], track2[0][2]])
            i = i - 1
            j = j - 1
        elif matrix[i, j] == matrix[i - 1, j - 1] - 1:
            i = i - 1
            j = j - 1
        elif matrix[i, j] == matrix[i - 1, j] - 1:
            i = i - 1
        elif matrix[i, j] == matrix[i, j - 1] - 1:
            j = j - 1
        else:
            break
    return a1

In [6]:
# List of all track ids in the dataset.

track_ids = pd.read_csv("preprocessed_list_of_track_ids.csv").values.tolist()

In [7]:
# This is the implementation if the Needleman-Wunsch algorithm.
# It takes one track as input and compares that track to every other track in the dataset coming after that track.


def Needleman_Wunsch(dataset, trackid, nr):
    track = dataset.loc[dataset["id"] == trackid]
    track = pd.DataFrame.drop_duplicates(track)
    print("track1: ", trackid)
    alignments = []
    for j in range(nr + 1, len(track_ids)):
        next_track = dataset.loc[dataset["id"] == track_ids[j][0]].values.tolist()
        matrix = create_matrix(track.values.tolist(), next_track)
        alignment = backtrack(matrix, track.values.tolist(), next_track)
        if len(alignment) > 3:
            alignments.extend(alignment)
    return alignments

In [8]:
# Create an overview of all points in the track and which other tracks has a matching point to that point.
# The output is a list of coordinate points where each has a corresponding list of track-ids.


def create_list_of_points(tracknr, alignments):

    track = dataset.loc[dataset["id"] == tracknr]
    track = pd.DataFrame.drop_duplicates(track)
    points = pd.DataFrame(columns=["lat", "lon", "ids"])
    align = pd.DataFrame(alignments, columns=["lat", "lon", "id"])

    for i in range(len(track)):
        lat = align.loc[align["lat"] == track.iloc[i]["lat"]]
        points.loc[i] = [
            track.iloc[i]["lat"],
            track.iloc[i]["lon"],
            lat["id"].values.tolist(),
        ]
    return points

In [9]:
# Convert the list of tracks-ids to number of tracks.
# The output is a list of coordinate points with a corresponding number of track ids matching with that point.


def convert_id_list_to_number_of_tracks(points):
    point_list = pd.DataFrame(points, columns=["lat", "lon", "ids"])
    for i in range(len(point_list)):
        distinct_point_list = list(set(point_list.loc[i]["ids"]))
        val = len(distinct_point_list)
        point_list.at[i, "ids"] = val
    return point_list

In [10]:
# The list of points from the previous function will be converted to result sequences.
# Each compinination of coordinate points with a high frequency will be awarded a value.
# The value is calculated by adding the length of the sequence, how many coordinate points are included,
# and the frequency of the sequence, the number of tracks matching the entire subtrack.
# The combination of coordinate points acchieving the highest value will be added returned.
# If there are multiple subsequences that are not overlapping acchiving a high value, all will be added to the final_records-list.


def gather_sequences(points):
    i = 0
    final_records = pd.DataFrame(columns=["polyline", "value"])
    for group in range(points.iloc[0]["group"], -1, -1):
        records = pd.DataFrame(columns=["polyline", "value"])
        sequence = []
        value = 0
        while i < len(points) and points.iloc[i]["group"] == group:
            if value == points.iloc[i]["ids"] or value == 0:
                sequence.append(
                    [
                        points.index[i],
                        points.iloc[i]["lat"],
                        points.iloc[i]["lon"],
                    ]
                )
                value = points.iloc[i]["ids"]
                i += 1
            else:
                sequence.sort()
                new_sequence = sequence.copy()
                if len(new_sequence) >= 3:
                    records.loc[len(records)] = [
                        new_sequence,
                        (len(new_sequence) * 1.0) + value,
                    ]
                value = points.iloc[i]["ids"]

        sequence.sort()
        new_sequence = sequence.copy()
        if len(new_sequence) >= 3:
            records.loc[len(records)] = [
                new_sequence,
                (len(new_sequence) * 1.0) + value,
            ]
        if len(records) < 2:
            final_records = pd.concat([final_records, records])
        else:
            highest_value = records["value"].idxmax()
            final_records = final_records.append(records.loc[highest_value])

    return final_records

In [14]:
# This is the fuction combining all the functions above.
# All the other functions are running for all tracks in the dataset.
# The output of this function is the complete result of the most trafficked roads using Smith-Waterman.


def hele_sullamitten(dataset):
    most_tracked_roads = pd.DataFrame(columns=["polyline", "value"])
    for i in range(len(track_ids)):
        track = track_ids[i][0]
        alignment = Needleman_Wunsch(dataset, track, i)
        points = create_list_of_points(track, alignment)
        new_points = convert_id_list_to_number_of_tracks(points)
        new_points = new_points[new_points.ids > 8]
        if not new_points.empty:
            new_points["group"] = (new_points.index.to_series().diff() > 2).cumsum()
            new_points.sort_values(by=["group", "ids"], ascending=False, inplace=True)
            records = gather_sequences(new_points)
            most_tracked_roads = pd.concat([most_tracked_roads, records])
            print(most_tracked_roads)
    return most_tracked_roads


res = hele_sullamitten(dataset)

track1:  T1
                                            polyline  value
0  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
track1:  T2
                                            polyline  value
0  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4  [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
track1:  T2334
track1:  T3
                                            polyline  value
0  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4  [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
track1:  T3463
track1:  T3569
track1:  T3985
track1:  T4
track1:  T61199
track1:  T612104
track1:  T7
                                            polyline  value
0  [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4  [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0  [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
track1:

                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
5   [[1, 41.148684, -8.585757000000001], [2, 41.14...   24.0
1   [[3, 41.15061000000001, -8.592597], [4, 41.150...   16.0
2   [[245, 41.14597499999999, -8.61363], [247, 41....   26.0
1   [[228, 41.145237, -8.616609], [229, 41.1452145...   19.0
0   [[219, 41.145993, -8.617905], [220, 41.1458265...   15.0
0   [[144, 41.152509, -8.626455], [145, 41.1525, -...   19.0
0   [[122, 41.1525426, -8.6301516], [123, 41.15253...   18.0
0   [[109, 41.15258550000002, -8.632298249999998],...   13.0
1   [[0, 41.163345, -8.633538000000001], [1, 41.16...   18.0
0   [[14, 41.157339, -8.

track1:  T305922
track1:  T31
                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
5   [[1, 41.148684, -8.585757000000001], [2, 41.14...   24.0
1   [[3, 41.15061000000001, -8.592597], [4, 41.150...   16.0
2   [[245, 41.14597499999999, -8.61363], [247, 41....   26.0
1   [[228, 41.145237, -8.616609], [229, 41.1452145...   19.0
0   [[219, 41.145993, -8.617905], [220, 41.1458265...   15.0
0   [[144, 41.152509, -8.626455], [145, 41.1525, -...   19.0
0   [[122, 41.1525426, -8.6301516], [123, 41.15253...   18.0
0   [[109, 41.15258550000002, -8.632298249999998],...   13.0
1   [[0, 41.163345, -8.633538000000001], [1, 41.16...  

                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
5   [[1, 41.148684, -8.585757000000001], [2, 41.14...   24.0
1   [[3, 41.15061000000001, -8.592597], [4, 41.150...   16.0
2   [[245, 41.14597499999999, -8.61363], [247, 41....   26.0
1   [[228, 41.145237, -8.616609], [229, 41.1452145...   19.0
0   [[219, 41.145993, -8.617905], [220, 41.1458265...   15.0
0   [[144, 41.152509, -8.626455], [145, 41.1525, -...   19.0
0   [[122, 41.1525426, -8.6301516], [123, 41.15253...   18.0
0   [[109, 41.15258550000002, -8.632298249999998],...   13.0
1   [[0, 41.163345, -8.633538000000001], [1, 41.16...   18.0
0   [[14, 41.157339, -8.

track1:  T46601439
track1:  T46191445
track1:  T47231461
                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
0   [[7, 41.148936000000006, -8.585775], [8, 41.14...   22.0
5   [[1, 41.148684, -8.585757000000001], [2, 41.14...   24.0
1   [[3, 41.15061000000001, -8.592597], [4, 41.150...   16.0
2   [[245, 41.14597499999999, -8.61363], [247, 41....   26.0
1   [[228, 41.145237, -8.616609], [229, 41.1452145...   19.0
0   [[219, 41.145993, -8.617905], [220, 41.1458265...   15.0
0   [[144, 41.152509, -8.626455], [145, 41.1525, -...   19.0
0   [[122, 41.1525426, -8.6301516], [123, 41.15253...   18.0
0   [[109, 41.15258550000002, -8.632298249999998],...   13.0
1   [[0, 41.163345, -8.63353

...

track1:  T115
track1:  T11993132
track1:  T120263154
track1:  T121763175
track1:  T122
track1:  T122863204
track1:  T122913215
track1:  T122273234
track1:  T123
track1:  T123173325
track1:  T123943331
track1:  T1231193348
track1:  T123503377
track1:  T123353382
track1:  T124413387
track1:  T124263404
track1:  T125
track1:  T126
track1:  T126373433
track1:  T130
                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
..                                                ...    ...
0   [[24, 41.164947000000005, -8.641755], [25, 41....   13.0
1   [[0, 41.145867, -8.61066], [1, 41.145983, -8.6...   25.0
6   [[3, 41.14589676923077, -8.61048623076923], [4...   96.0
1   [[31, 41.1461247272727

                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
..                                                ...    ...
1   [[33, 41.15353949999997, -8.608677750000005], ...   21.0
0   [[8, 41.153895000000006, -8.613189], [9, 41.15...   14.0
1   [[6, 41.14773, -8.620216200000002], [7, 41.147...   33.0
10  [[38, 41.14807199999999, -8.612406000000004], ...  169.0
2   [[16, 41.1472215, -8.6088555], [17, 41.147433,...   27.0

[99 rows x 2 columns]
track1:  T167
track1:  T167194141
track1:  T167314144
track1:  T168644170
track1:  T168524176
track1:  T169754184
track1:  T170
track1:  T170204216
track1:  T171
track1:  T171874234
track1:  T171354258
track1:  T172
track1:  T1721064277
track1: 

track1:  T205555271
track1:  T206425277
track1:  T206955344
track1:  T2061115361
track1:  T206815370
track1:  T206895378
track1:  T207
track1:  T207215401
track1:  T207665428
track1:  T207675434
track1:  T209
track1:  T210625484
track1:  T211385495
track1:  T21115505
track1:  T211145516
track1:  T211885529
track1:  T211575539
track1:  T211565544
track1:  T211155584
track1:  T21375604
track1:  T214
                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
..                                                ...    ...
0   [[104, 41.1692025, -8.6062125], [105, 41.16936...   17.0
1   [[64, 41.162913, -8.606772], [65, 41.163012, -...   42.0
1   [[34, 41.145939, -8.612910000000001], [35, 41.

                                             polyline  value
0   [[3, 41.14892699999999, -8.585730000000002], [...   24.0
4   [[3, 41.145909, -8.61075], [4, 41.14605, -8.61...   29.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   20.0
0   [[0, 41.14467, -8.606529], [1, 41.144724, -8.6...   15.0
..                                                ...    ...
3   [[55, 41.148009, -8.610894000000002], [56, 41....   32.0
2   [[2, 41.148216, -8.624079], [7, 41.148216, -8....   43.0
0   [[17, 41.148107, -8.610863000000002], [18, 41....   23.0
0   [[32, 41.147571, -8.617667999999998], [33, 41....   15.0
4   [[1, 41.147811, -8.620144714285715], [2, 41.14...   95.0

[128 rows x 2 columns]
track1:  T241
track1:  T242
track1:  T242676972
track1:  T24236983
track1:  T243
track1:  T243566991
track1:  T24336999
track1:  T245
track1:  T245647041
track1:  T246
                                             polyline  value
0   [[3, 41.148

track1:  T292417958
track1:  T293287979
track1:  T293237987
track1:  T29318000
track1:  T294
track1:  T294678040
track1:  T295
track1:  T296
track1:  T297
track1:  T298928122
track1:  T3001008148
track1:  T301
track1:  T301818196
track1:  T302
track1:  T30218208
track1:  T302138215
track1:  T304268236
track1:  T305
track1:  T3051138254
track1:  T305238258
track1:  T305238264
track1:  T306
track1:  T306488278
track1:  T307
track1:  T30738352
track1:  T311
track1:  T311688385
track1:  T311888388
track1:  T311608394
track1:  T311318402
track1:  T3121078422
track1:  T313288443
track1:  T314748463
track1:  T31438472
track1:  T315238482
track1:  T315618490
track1:  T316
track1:  T316148501
track1:  T317228519
track1:  T319
track1:  T319808578
track1:  T320
track1:  T320538598
track1:  T320318607
track1:  T320198616
track1:  T32008621
track1:  T321728634
track1:  T32188640
track1:  T322
track1:  T322818678
track1:  T323198704
track1:  T324
track1:  T324238722


In [15]:
pd.DataFrame.to_csv(res, "./Needleman_Wunsch_result.csv", index=True)